<a href="https://colab.research.google.com/github/rifkybujana/KOPSI/blob/Research%2FModel/FacQA_Analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## INSTALLATION

In [1]:
!pip install tokenizers

     |████████████████████████████████| 3.3MB 4.0MB/s 


In [2]:
!pip install transformers

     |████████████████████████████████| 2.5MB 2.9MB/s 
     |████████████████████████████████| 901kB 13.9MB/s 


## CODE

Import required libraries

In [3]:
import torch
import pandas as pd
import numpy as np

from ast import literal_eval

from transformers import DistilBertTokenizerFast
from transformers import DistilBertForQuestionAnswering

from transformers import AdamW

from torch.utils.data import DataLoader

### Analyzing FacQA dataset

import train and eval dataset

In [4]:
train = pd.read_csv('train_preprocess.csv')
valid = pd.read_csv('val_preprocess.csv')

analalyze train dataset

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2495 entries, 0 to 2494
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   question   2495 non-null   object
 1   passage    2495 non-null   object
 2   seq_label  2495 non-null   object
dtypes: object(3)
memory usage: 58.6+ KB


In [6]:
train.describe()

,question,passage,seq_label
count,2495,2495,2495
unique,2412,1216,2077
top,"['Siapa', 'nama', 'presiden', 'Indonesia', 'se...","['Pemilu', 'pertama', 'multikandidat', 'ini', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
freq,7,17,6


In [7]:
train.head()

,question,passage,seq_label
0,"['Kelompok', 'apakah', 'yang', 'menyatakan', '...","['Lewat', 'telepon', 'ke', 'kantor', 'berita',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,"['Di', 'pulau', 'manakah', ',', 'PT', 'Mutiara...","['Sejak', 'tahun', '1980', '-', 'an', 'di', 'K...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,"['Apakah', 'warna', 'pasir', 'pada', 'pesisir'...","['Titik', 'kedua', 'adalah', 'Pantai', 'Baturu...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,"['Apa', 'nama', 'perusahaan', 'China', 'yang',...","['Sichuan', ',', 'Kompas', '-', 'Pemerintah', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,"['Siapa', 'yang', 'akan', 'memindahkan', 'Patu...","['Jakarta', ',', 'Kompas', '-', 'Patung', 'Kar...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [8]:
type(train['question'][0])

str

As you can see, the dataset data type are string, but we want them to be a list, not a string, we need to remove all the brackets, quotation marks, and the coma's. separate every word, and convert it into an array using `literal_eval` function

In [9]:
train['question'] = train['question'].apply(literal_eval)
train['passage'] = train['passage'].apply(literal_eval)
train['seq_label'] = train['seq_label'].apply(literal_eval)

In [10]:
valid['question'] = valid['question'].apply(literal_eval)
valid['passage'] = valid['passage'].apply(literal_eval)
valid['seq_label'] = valid['seq_label'].apply(literal_eval)

after applied all the dataset values with the `literal_eval` function, we can see, that the dataset datatype changed into a list

In [11]:
print(f"""train dataset data type: {type(train['question'][0])}""")
print(f"""valid dataset data type: {type(valid['question'][0])}""")

train dataset data type: <class 'list'>
valid dataset data type: <class 'list'>


But the data in `seq_label` collumn, is still hard to understand, what does it mean, and how the model should read it, so we need to convert it.

In [13]:
print(train['seq_label'][0])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [14]:
def seq_to_index(seq):
    result = []
    for i in range(len(seq)):
        if seq[i] != 'O':
            result.append(i)

    result = [result[0], result[-1]]

    return result

train['seq_label'] = train['seq_label'].apply(seq_to_index)
valid['seq_label'] = valid['seq_label'].apply(seq_to_index)

In [17]:
train.head()

,question,passage,seq_label
0,"[Kelompok, apakah, yang, menyatakan, bertanggu...","[Lewat, telepon, ke, kantor, berita, lokal, Cu...","[10, 11]"
1,"[Di, pulau, manakah, ,, PT, Mutiara, Indopearl...","[Sejak, tahun, 1980, -, an, di, Kota, Bau-Bau,...","[30, 31]"
2,"[Apakah, warna, pasir, pada, pesisir, pantai, ...","[Titik, kedua, adalah, Pantai, Baturumah, sepa...","[19, 20]"
3,"[Apa, nama, perusahaan, China, yang, bekerja, ...","[Sichuan, ,, Kompas, -, Pemerintah, Indonesia,...","[9, 12]"
4,"[Siapa, yang, akan, memindahkan, Patung, Karti...","[Jakarta, ,, Kompas, -, Patung, Kartini, di, J...","[18, 20]"


In [18]:
valid.head()

,question,passage,seq_label
0,"[Apa, nama, latin, dari, harimau, Sumatera, ?]","[Selain, orangutan, ,, satwa, yang, berada, TN...","[14, 16]"
1,"[Berapa, penumpang, yang, turut, dalam, penerb...","[Pesawat, dengan, nomor, penerbangan, GA, -, 1...","[9, 10]"
2,"[Kepala, Badan, apakah, Mohamed, ElBaradei, ?]","[Pernyataan, tersebut, disampaikan, Larijani, ...","[14, 20]"
3,"[Berapa, kalikah, Lin, Dan, gagal, memainkan, ...","[Tak, seperti, yang, diharapkan, ,, partai, im...","[33, 34]"
4,"[Apakah, nama, fakultas, tempat, Drh, Yuda, He...","[Yogyakarta, ,, Kompas, -, Drh, Yuda, Heru, Fi...","[11, 13]"


In [19]:
train.rename(columns = {'seq_label':'answer_index'}, inplace = True)
valid.rename(columns = {'seq_label':'answer_index'}, inplace = True)

In [21]:
train.rename(columns = {'passage':'context'}, inplace = True)
valid.rename(columns = {'passage':'context'}, inplace = True)

In [22]:
print(train.columns)
print(valid.columns)

Index(['question', 'context', 'answer_index'], dtype='object')
Index(['question', 'context', 'answer_index'], dtype='object')


In [23]:
def index_to_answer(context, index):
    result = []
    for i in range(index[0], index[1] + 1):
        result.append(context[i])

    result = ' '.join(result)

    return result

In [27]:
def convert_index(context, index):
    result = []
    for i in range(len(context)):
        result.append(index_to_answer(context[i], index[i]))

    return result

In [29]:
train_answer = convert_index(train['context'], train['answer_index'])
valid_answer = convert_index(valid['context'], valid['answer_index'])

In [30]:
train['answer'] = train_answer
valid['answer'] = valid_answer

In [31]:
train.head()

,question,context,answer_index,answer
0,"[Kelompok, apakah, yang, menyatakan, bertanggu...","[Lewat, telepon, ke, kantor, berita, lokal, Cu...","[10, 11]",Hezb-ul Mujahedeen
1,"[Di, pulau, manakah, ,, PT, Mutiara, Indopearl...","[Sejak, tahun, 1980, -, an, di, Kota, Bau-Bau,...","[30, 31]",Pulau Makassar
2,"[Apakah, warna, pasir, pada, pesisir, pantai, ...","[Titik, kedua, adalah, Pantai, Baturumah, sepa...","[19, 20]",putih halus
3,"[Apa, nama, perusahaan, China, yang, bekerja, ...","[Sichuan, ,, Kompas, -, Pemerintah, Indonesia,...","[9, 12]",Sinopec Southwest Petroleum Biro
4,"[Siapa, yang, akan, memindahkan, Patung, Karti...","[Jakarta, ,, Kompas, -, Patung, Kartini, di, J...","[18, 20]",Pemerintah DKI Jakarta


In [32]:
valid.head()

,question,context,answer_index,answer
0,"[Apa, nama, latin, dari, harimau, Sumatera, ?]","[Selain, orangutan, ,, satwa, yang, berada, TN...","[14, 16]",Panthera tigris sumatrensis
1,"[Berapa, penumpang, yang, turut, dalam, penerb...","[Pesawat, dengan, nomor, penerbangan, GA, -, 1...","[9, 10]",84 penumpang
2,"[Kepala, Badan, apakah, Mohamed, ElBaradei, ?]","[Pernyataan, tersebut, disampaikan, Larijani, ...","[14, 20]",Badan Tenaga Atom Internasional ( IAEA )
3,"[Berapa, kalikah, Lin, Dan, gagal, memainkan, ...","[Tak, seperti, yang, diharapkan, ,, partai, im...","[33, 34]",15 kali
4,"[Apakah, nama, fakultas, tempat, Drh, Yuda, He...","[Yogyakarta, ,, Kompas, -, Drh, Yuda, Heru, Fi...","[11, 13]",Fakultas Kedokteran Hewan
